![logo](../picture/license_header_logo.png)

> **Copyright (c) 2020-2021 CertifAI Sdn. Bhd.**<br>
 <br>
This program is part of OSRFramework. You can redistribute it and/or modify
<br>it under the terms of the GNU Affero General Public License as published by
<br>the Free Software Foundation, either version 3 of the License, or
<br>(at your option) any later version.
<br>
<br>This program is distributed in the hope that it will be useful,
<br>but WITHOUT ANY WARRANTY; without even the implied warranty of
<br>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
<br>GNU Affero General Public License for more details.
<br>
<br>You should have received a copy of the GNU Affero General Public License
<br>along with this program.  If not, see <http://www.gnu.org/licenses/>.
<br>

Authored by: [BK Yeoh](boonkhai.yeoh@certifai.ai)

# Exercise
## Introduction
The data we'll be using is a combined curated dataset of COVID-19 Chest X-ray images obtained by collating 15 publically available datasets. [source](https://data.mendeley.com/datasets/9xkhgts2s6/1)

The dataset contains 1281 COVID-19 X-Rays, 3270 Normal X-Rays, 1656 viral-pneumonia X-Rays, and 3001 bacterial-pneumonia X-Rays. The 4 classes are:
* COVID-19
* Normal
* Pneumonia-Bacterial
* Pneumonia-Viral

Use the model in the previous `image_classifier_exercise` as the base model for this exercise. Feel free to use the helper testing function `utils.Toolbox` to validate your model correctness.

You are required:
* Save your model using:
    * Python Pickle
    * Joblib
    * PyTorch TorchScript
* Save your model format in:
    * ONNX
    * Protocol Buffers
    * JSON format
* Pass the File test
* Pass the Accuracy Test


In [ ]:
import numpy as np
from pathlib import Path
import torchvision
from torchvision import transforms,datasets,models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.onnx
from time import time
import warnings
import os
import utils

## Helper Function

In [ ]:
# Generate folder to store the model
gen_model_path = 'generated_model/exercise'
dir_list = ['pickle','joblib','torchscript','onnx','protocol_buffers','json'] 
utils.folder_generator(gen_model_path,dir_list)

In [ ]:
source = 'https://s3.eu-central-1.wasabisys.com/certifai/deployment-training-labs/xray_image_classification-20210604T123548Z-001.zip'
target = '../resources/data/'
filename = 'xray_image_classification.zip'
utils.download(source, target, filename,True)

model_url = "https://s3.eu-central-1.wasabisys.com/certifai/deployment-training-labs/models/xray_classifier_state_dict.pt"
modelname = "xray_classifier_state_dict.pt"
utils.download(model_url, gen_model_path, modelname)

In [ ]:
# The data is located in the data folder
datadir = Path().resolve().parent/'resources/data/'
dirtytestdir = datadir/'xray_image_classification/test/'

In [ ]:
val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

In [ ]:
model = models.squeezenet1_0(pretrained=True)
# Freezing the parameters
for param in model.parameters():
    param.requires_grad = False
    
# Replace a custom classifier to fit our data
torch.manual_seed(123)

num_output = 4
model.classifier = nn.Sequential(nn.Dropout(0.5),
                                 nn.Conv2d(512, num_output, 1, 1),
                                 nn.ReLU(),
                                 nn.AvgPool2d(13, 1))

model.load_state_dict(torch.load(os.path.join(gen_model_path,modelname)))
model

In [ ]:
'''
Arguments 
model: base model define by user
dirtytestdir: test dataset directory
val_transform: image transforms
'''
toolbox = utils.Toolbox(model,dirtytestdir,val_transform)

## Save & Load the Model using Python Pickle
Save and load your model <br>
`model file path = '../generated_model/exercise/pickle/xray_image_classification.pt'`

In [ ]:
import pickle

# save the model to disk
def pickle_func(directory = gen_model_path):
    # YOUR CODE HERE
    return loaded_model_pk

In [ ]:
toolbox.test(os.path.join(gen_model_path,dir_list[0]+'/'+"xray_image_classification.pt"),
     pickle_func())

## Save & Load the Model using Joblib
Save and load your model <br>
`model file path = '../generated_model/exercise/joblib/xray_image_classification.pt'`<br>

In [ ]:
import joblib

# save the model to disk
def joblib_func(directory = gen_model_path):
    # YOUR CODE HERE
    return loaded_model_jb

In [ ]:
toolbox.test(os.path.join(gen_model_path,dir_list[1]+'/'+"xray_image_classification.pt"),
     joblib_func())

## Save & Load the Model using TorchScript
Use `torch.jit.script` to save you model and `torch.jit.load` to load it.<br>
`model file path = '../generated_model/exercise/torchscrip/xray_image_classification.pt'`<br>

In [ ]:
def torch_func(directory = gen_model_path):
    # YOUR CODE HERE
    return loaded_script_model

In [ ]:
toolbox.test(os.path.join(gen_model_path,dir_list[2]+'/'+"xray_image_classification.pt"),
     torch_func())

## Save Model in ONNX format
Save you model in ONNX format
`model file path = '../generated_model/exercise/onnx/xray_image_classification.onnx'`<br>

In [ ]:
x = torch.randn(10, 3, 224, 224, requires_grad=True)
# Export the model
# YOUR CODE HERE

In [ ]:
toolbox.test(os.path.join(gen_model_path,dir_list[3]+'/'+"xray_image_classification.onnx"))

## Save Model in Protocol Buffers
Save you model in `.pb` format<br>
`model file path = '../generated_model/exercise/protocol_buffers'`<br>

In [ ]:
from onnx_tf.backend import prepare
import onnx
warnings.filterwarnings("ignore")
# YOUR CODE HERE
!saved_model_cli show --dir {pb_filepath} --all

In [ ]:
toolbox.test(os.path.join(gen_model_path,dir_list[4]+'/'+"saved_model.pb"))

## Save model in Json format
Save you model in `.json` format<br>
`model file path = '../generated_model/exercise/json/xray_image_classification.json'`<br>

In [ ]:
# YOUR CODE HERE

In [ ]:
toolbox.test(os.path.join(gen_model_path,dir_list[5]+'/'+"xray_image_classification.json"))